# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System
#### Jean Paul Barddal, M.Sc.


### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Assuming that a student will either (i) pass or (ii) fail to graduate, this should be considered a binary classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [19]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [20]:
# Let's look at the header
# print student_data.describe()
# print student_data.head()

# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features, i.e. the number of attributes minus 1 (the class feature)
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
pass_students = student_data.loc[student_data.passed == 'yes', 'passed']
n_passed = pass_students.shape[0]

# Calculate failing students (the complement of those that passed)
n_failed = n_students - n_passed

# Calculate graduation rate
grad_rate = float(100 * n_passed)/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [21]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [22]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

** As shown before, the data set is somewhat unbalanced, and thus, instead of simply splitting the data, it would be nice that both training and test sets would have the same class distribution. That's why StratifiedShuffleSplit was used instead of the conventional train_test_split.**

In [23]:
# Import any additional functionality you may need here
from sklearn.model_selection import StratifiedShuffleSplit

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
splits = StratifiedShuffleSplit(n_splits=1, test_size=float(num_test)/(n_students), \
                                train_size=num_train, random_state=15)
# print splits
for train_index, test_index in splits.split(X_all, y_all):
    X_train, X_test = X_all.loc[train_index], X_all.iloc[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** Determining which algorithms should be applied to this data set is not trivial. This occurs due to the fact that the dataset is unbalanced (aprox 67%-33%), has very few instances (395) and a fair amount of attributes (30), which may cause some algorithms to fall into the "curse of dimensionality" [1].

1- **(Naive Bayes)** Given these characteristics, the first choice would be the Naive Bayes classifier, which is known for being a good choice on high-dimensional data, is fast to compute and if widely used in real-world applications, such as: text classification [2], emotion detection [3] and many more. As strengths of Naive Bayes, it is mentioned: easy to train and implement, low model complexity. And as cons: no variable dependency, which is unjustified for most of real-world data. Multinomial NB is used here since all attributes were converted into categorical ones, so no approximation such as the GaussianNB is needed here.

2- **(K-Nearest Neighbors)** kNN is a family of lazy learners that classify instances given the k closest records stored in buffer. The definition of close means that a distance metric is used to determine how similar two records are. kNN is so widely used that is hard to list all of its applications, so a few are as follows: heart disease classification [4] and stock price prediction [5, 6]. Even though kNN is strongly subject to the curse of dimensionality, it is able to learn complex decision borders, and thus, it should be evaluated. Furthermore, kNN (and some of its variations) obtained quite successful accuracy rates in very high-dimensional data streams, as shown in [6]. Strengths of kNN include: simple implementation, flexibility to different distance metrics, is able to depict complex decision borders and handles multi-class problems (even though it's not the case here). As cons: is problematic in high-dimensional problems due to the curse of dimensionality, is computationally expensive for queries in the $O(nd)$ order, where $n$ is the number of training points and $d$ their dimensionality and requires a meaningful distance metric.


3- **(Decision Tree)** Decision trees represent a family of algorithms that learn classification models by performing multiple feature selections until a purity level amongst a subset of instances is obtained. Decision trees are interesting since their final model is easily readable and is able to overcome both irrelevant and redundant features quite succesfully [7]. Aditionally, since the final model is readable, it can be verified by experts on the field to check if the used attributes and the tested values in each branch of the tree make sense. Applications of decision trees are also colossal, broadening agriculture to even software development [8]. Pros of decision trees include: low (or no) need to select features before training, readability (since they are white-boxes), and low cost for model construction. As cons: instability (since small variations in the training input lead to different trees, which is an appropriate trait for ensembles), trees tend to grow unwiedly given the number of trees, which is problematic since larger trees are hard to read and hard to explain.



[1] http://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/

[2] https://web.stanford.edu/class/cs124/lec/naivebayes.pdf

[3] http://www.ifp.illinois.edu/~iracohen/publications/icpr02.pdf

[4] http://www.ijbhtnet.com/journals/Vol_3_No_3_March_2013/4.pdf

[5] http://www.igi-global.com/article/advances-on-concept-drift-detection-in-regression-tasks-using-social-networks-theory/124879

[6] http://link.springer.com/chapter/10.1007%2F978-3-319-46227-1_9

[7] http://ieeexplore.ieee.org/document/7372247/?arnumber=7372247

[8] http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [24]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [25]:
# Import the three supervised learning models from sklearn
#http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB
from sklearn.naive_bayes import MultinomialNB 
#http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
from sklearn.neighbors import KNeighborsClassifier 
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier 

# Initialize the three models
clf_A = MultinomialNB()
clf_B = KNeighborsClassifier()
clf_C = DecisionTreeClassifier(random_state=15)

# Set up the training set sizes

# I found these variables to be useless, 
# that's why I commented them out
# X_train_100 = 100
# y_train_100 = 100

# X_train_200 = 200
# y_train_200 = 200

# X_train_300 = 300
# y_train_300 = 300

# Execute the 'train_predict' function for each classifier and each training set size
classifiers = [clf_A, clf_B, clf_C]
sizes = [100, 200, 300]
for clf in classifiers:
    for nTrain in sizes:
        train_predict(clf, X_train[:nTrain], y_train[:nTrain], X_test, y_test)

Training a MultinomialNB using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.7794.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8058.
Training a MultinomialNB using a training set size of 200. . .
Trained model in 0.0009 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8042.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8058.
Training a MultinomialNB using a training set size of 300. . .
Trained model in 0.0018 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8056.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.8235.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0029 seconds.
F1 score for training set: 0.7917.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.7571.
Training a KNeighborsClassifier using a training set size of 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Multinomial NB**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0003            |      0.0003         |   0.7794               | 0.8058 |
| 200               |        0.0003           |0.0002 |0.8042|0.8058|
| 300               |0.0004 |0.0002 |0.8056|    **0.8235**      |

** Classifer 2 - kNN**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0018 |0.0017 |0.7917| 0.7571|
| 200               |0.0032 |0.0020 |0.8132|0.7737|
| 300               |0.0076 |0.0031 |0.8545|     **0.8108**     |

** Classifer 3 - Decision Tree Classifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0002 |0.0002|1.0000|**0.7407**|
| 200               |0.0003 |0.0002|1.0000|0.6667|
| 300               |0.0005 |0.0002|1.0000|0.7077|

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** The results obtained are very clear on stating which classifier should be used in the system. Even though I believe processing time is not critical in this task, since these predictions will be performed maybe once per month or year, achieving higher accuracy rates is critical. Thankfully, the results obtained by Multinomial Naive Bayes present the best overall F1 scores and a very small prediction time, showing that a simple model is beneficial in both accuracy and time consumption. Nonetheless if we imagine that the data set would evolve year after year with the input of new students' data, Naive Bayes would be even more recomended, since it presents the best prediction times.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** The proposed model learns, by computing probabilities, how likely (in probability terms) a student is to graduate or fail given some of his performance indicators. This model is based on data from previous students, which either graduated or failed to graduate in the previous years. After the model is constructed, it can be used to predict whether a student will graduate or not. Upon the input of a student's indicators, the model will compute a probability to both of the possible outcomes: graduation or failure. The final output will then be an answer in the "student is likely to graduate with X% chance" or "student it likely to graduate with Y% chance" form. Given these results, one is able to take preventive actions towards students that are likely to fail.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [26]:
# Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# I will tune all three classifiers in here, 
# so I wrote some extra code to tune each classifier
# since each one of them uses different parameters

def getTuningParametersNB():
    alphas = np.arange(0.0, 1.05, 0.05)
    params = {'alpha': alphas, 'fit_prior': [True, False]}
    return params
def getTuningParametersKNN():
    kValues = [i for i in range(1, 10, 2)]
    params = {'n_neighbors': kValues,
                  'p': [1, 2, 3]}
    return params
def getTuningParametersDT():
    params = {'criterion': ('gini', 'entropy'),
                  'splitter': ('best', 'random')}
    return params
    
def tuneClassifier(clf):
    clfName = clf.__class__.__name__
    parameters = []
    if type(clf) is MultinomialNB:
        parameters = getTuningParametersNB()
    if type(clf) is KNeighborsClassifier:
        parameters = getTuningParametersKNN()
    if type(clf) is DecisionTreeClassifier:
        parameters = getTuningParametersDT()
    print 'Tuning {}'.format(clfName)
    f1_scorer = make_scorer(f1_score, pos_label='yes')
    grid_obj = GridSearchCV(estimator=clf, param_grid=parameters,\
                        n_jobs=4, cv=10,\
                        scoring=f1_scorer)
    grid_obj.fit(X_train, y_train)
    clf = grid_obj.best_estimator_
    print "\tTuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "\tTuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

    
# print y_train    
for clf in classifiers:
    tuneClassifier(clf)


Tuning MultinomialNB
Made predictions in 0.0005 seconds.
	Tuned model has a training F1 score of 0.8000.
Made predictions in 0.0003 seconds.
	Tuned model has a testing F1 score of 0.8235.
Tuning KNeighborsClassifier
Made predictions in 0.0315 seconds.
	Tuned model has a training F1 score of 0.8387.
Made predictions in 0.0102 seconds.
	Tuned model has a testing F1 score of 0.7891.
Tuning DecisionTreeClassifier
Made predictions in 0.0004 seconds.
	Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0004 seconds.
	Tuned model has a testing F1 score of 0.7385.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** All three of the learning algorithms were tuned. Unfortunately, no improvements were found, as can be seen in the table below. It is important to highlight the results obtained by kNN, which are somewhat lower than the one obtained without tuning. It is also, nonetheless, difficult to compare these results directly, since the optimization of the parameters adopts the average of a 10-fold cross-validation that is performed within the grid search.

### Results Comparison Before and After Tuning

| Algorithm    | F1 Test Score Before Tuning | F1 Score After Tuning |
| :--------:   | :-------: | :-------------: |
| Naive Bayes  |**0.8235** |**0.8235**|
| KNN          |**0.8108** |0.7891|
| Decision Tree|**0.7407** |0.7385|